<a href="https://colab.research.google.com/github/tuonglab/BIOL3003_workshop/blob/master/notebook/BIOL3003_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UQ BIOL3003 Workshop: Introduction to Single-Cell RNA Sequencing Analysis with Python

<span style="color:red; font-weight:bold; font-size:24px">This is an introductory exercise to how to performing cell-type annotation in single-cell RNA-sequencing analysis with Python.</span>

Students will work in groups of 2-4. Each group will be provided with a single-cell dataset of mouse spleens from 12-week old mice that were sent to the International Space Station by NASA (space; n=4) versus mice that stayed at ground control (earth; n=4). The main exercise is to <span style="color:red; font-weight:bold">annotate the major immune cell types found in spleen</span> in Python using [`Scanpy`](https://scanpy.readthedocs.io/en/stable/), the toolkit for analysing single-cell gene expression data. Time permitting, <span style="color:red; font-weight:bold">we will also quickly explore if space travel led to differential abundance of the immune cell types</span>. The data has undergone initial pre-processing to enable the cell-type annotation exercise.

<span style="color:red; font-weight:bold; font-size:24px">Unless otherwise instructed, for every step below, the student will simply need to run the entire CODE CHUNK by clicking the run/play “►” button to execute the code.</span>

### STEP 1: Installing pre-requisite software and downloading data

In [ ]:
# install scanpy
!pip install scanpy[leiden]
# clone the repository so that we have all the data and notebooks ready to go
!git clone https://github.com/tuonglab/BIOL3003_workshop.git


### STEP 2: Loading up software

In [1]:
import os
import scanpy as sc

# change to working directory
os.chdir("BIOL3003_workshop")

### STEP 3: Reading the data and inspect the object

We will read the processed data and assign it to a variable called `adata`. We will then inspect the object to understand the structure of the data.

In [ ]:
adata = sc.read_h5ad("data/mouse_spleen_processed.h5ad")
# Print the summary of the object
adata

<span style="color:red; font-weight:bold; font-size:36px">Question 1. How many cells are there in the dataset?</span>


### STEP 4: Visualising the data - Clusters

We will visualise the data using a UMAP plot. Don't need to worry about what UMAP stands for. UMAP is a dimensionality reduction technique that is commonly used in single-cell analysis to visualise high-dimensional data in 2D or 3D. It is particularly useful for visualising clusters of cells and gene expression patterns. Here, every single dot represents a single cell in the dataset.

We will first visualise the data coloured by the clusters that were identified during the clustering step in the pre-processing.


In [ ]:
sc.pl.umap(adata, color="leiden")

Sometimes, it is difficult to connect the clusters on the UMAP to the legend. We will add a few more arguments to the `sc.pl.umap` function to make the plot more informative.

In [ ]:
sc.pl.umap(adata, color="leiden", legend_loc="on data", legend_fontoutline=2)

<span style="color:red; font-weight:bold; font-size:36px">Question 2. How many clusters have been found in the dataset?</span>

### STEP 5: Visualising gene expression with UMAP - I

We will now visualise the data coloured by the expression of a gene of interest. We will use the gene `Cd4`, which is a gene that is commonly used to identify CD4 T cells


In [ ]:
sc.pl.umap(adata, color="Cd4")

The size of the dots in the plot are a bit too small. So we will increase the size of the dots by setting the `size` argument in the `sc.pl.umap` function to `20`.

In [ ]:
sc.pl.umap(adata, color="Cd4", size=20)

### STEP 6: Visualising gene expression with UMAP - II

Now, let's repeat the same as step 5 but with a different gene, `Cd8a`, which is a gene that is commonly used to identify CD8 T cells.

In [ ]:
sc.pl.umap(adata, color="Cd8a", size=20)

<span style="color:red; font-weight:bold; font-size:36px">Question 3. From steps 5 and 6, which cluster numbers probably correspond to CD4 T cells and CD8 T cells?</span>

### STEP 7: Visualising gene expression with dot plots

Another way we can visualise the data is to plot the expression of the genes in a dot plot across the clusters. We will use the `sc.pl.dotplot` function to do this. The size of the circles in the dot plot corresponds to the fraction of cells in the cluster that express the gene. The colour of the circles corresponds to the average expression of the gene in the cluster. Therefore, the larger the circle and higher or darker the colour on the gradient, the more highly expressed the gene is in the cluster. 

We will plot the expression of the genes `Cd4` and `Cd8a` in the dot plot.

Try changing the `color_map` from 'Blues' to 'viridis' or 'magma' to see how the plot changes.

In [ ]:
sc.pl.dotplot(
    adata, ["Cd4", "Cd8a"], groupby="leiden", standard_scale="var", color_map="Blues"
)

<span style="color:red; font-weight:bold; font-size:36px">Question 4. Which cluster numbers probably correspond to CD4 T cells and CD8 T cells? Does this match your answer from question 3?</span>

### STEP 7: Visualising the top marker genes of each cluster with dot plots

Marker genes are genes that are highly expressed in a particular cluster compared to other clusters. This is typically performed using statistical tests such as the Wilcoxon rank-sum test. This has been done for you and we can visualise the top marker genes using the `sc.pl.rank_genes_groups_dotplot`function.

Let's look at the top `3` marker genes, limiting to only genes that are at least 1 log-fold higher in expression in the cluster compared to other clusters. 

Try changing the `n_genes` argument to `5` to see the top 5 marker genes.

In [ ]:
sc.pl.rank_genes_groups_dotplot(
    adata, n_genes=3, color_map="Blues", standard_scale="var", min_logfoldchange=1
)

In [ ]:
sc.pl.rank_genes_groups(adata, sharey=False)

<span style="color:red; font-weight:bold; font-size:36px">Question 5. What are the top 3-5 marker genes that could be useful to annotate the following cell types in mouse spleen?</span>

I will give you a hint for the first cell type:
    
    - Haematopoietic stem cells
        Cd34
        Spn # Also known as CD43
        Kit # Also known as CD117

Fill in the rest of the cell types:

    - Developing B cells

    - B cells

    - Plasma cells

    - CD4 T cells

    - CD8 T cells

    - NK cells
    
    - Macrophages

    - Monocytes

    - Neutrophils
    
    - Dendritic cells

    - Plasmacytoid Dendritic cells

    - Erythrocytes

    - Platelets

    - Endothelial cells/Fibroblasts/stromal cells

    

### STEP 8: Selecting the best marker genes for each cell type

Let's visualise your response to question 5 as a dot plot to see if the marker genes are indeed specific to the cell types.

Fill in the `marker_genes_dict` dictionary with at least 3 genes that can potentially be used as marker genes for each cell-type and then run the code. I have filled in the first one for you.

In [ ]:
marker_genes_dict = {
    "Haematopoietic stem cells": ["Cd34", "Spn", "Kit"],
    "Developing B cells": [],
    "B cells": [],
    "Plasma cells": [],
    "CD4 T cells": [],
    "CD8 T cells": [],
    "NK cells": [],
    "Macrophages": [],
    "Monocytes": [],
    "Neutrophils": [],
    "Dendritic cells": [],
    "Plasmacytoid Dendritic cells": [],
    "Erythrocytes": [],
    "Platelets": [],
    "Endothelial cells": [],
    "Fibroblasts/stromal cells": [],
}
sc.pl.dotplot(
    adata, marker_genes_dict, groupby="leiden", standard_scale="var", color_map="Blues"
)

### STEP 9: Annotating the cell types

We will now annotate the cell types in the dataset using the marker genes that you have selected. We will create a dictionary called `cell_type_dict` where the keys are the cluster numbers and the values are the cell types that you have annotated. When you run the code, the function will annotate the cell type and plot the UMAP coloured by the cell types you have annotated. Rerun this code chunk until you are satisfied with the annotation.

I have filled in cluster `8` for you. Students are to fill in the rest of the clusters.


In [ ]:
cell_type_dict = {
    "0": "",
    "1": "",
    "2": "",
    "3": "",
    "4": "",
    "5": "",
    "6": "",
    "7": "",
    "8": "B cells",
    "9": "",
    "10": "",
    "11": "",
    "12": "",
    "13": "",
    "14": "",
    "15": "",
    "16": "",
    "17": "",
    "18": "",
    "19": "",
    "20": "",
    "21": "",
    "22": "",
    "23": "",
    "24": "",
    "25": "",
    "26": "",
    "27": "",
}
adata.obs["cell_type"] = adata.obs["leiden"].map(cell_type_dict)
sc.pl.umap(adata, color="cell_type")

<span style="color:blue; font-weight:bold; font-size:24px">BONUS EXERCISE FOR STUDENTS</span>

Now that we have annotated the cell types, we can explore if space travel led to differential abundance of the immune cell types. Let's first tabulate how many cells of each cell type are present in the dataset, split by mice that were in space or on ground control.

We will use the `pandas` library to do some data frame manipulation that will let us tabulate the proportions of each cell type, split by mice that were in space or on ground control.


In [ ]:
import pandas as pd

data = pd.crosstab(adata.obs["sample"], adata.obs["cell_type"])
data

Let's express this as a percentage of the total number of cells in each mouse.

In [ ]:
proportion = data.apply(lambda r: r / r.sum() * 100)
proportion

Let's plot the proportions as barplots. We first need to rehape the data 

In [ ]:
# create a new column to split the conditions
proportion["group"] = proportion.index.str.split("_").str[0]
proportion.plot(kind="box", by="group", subplots=True, layout=(5, 4), figsize=(10, 20))

Let's perform a simple t-test to see if the proportions of each cell type are different between the space and ground control mice.

In [ ]:
from scipy.stats import ttest_ind

for cell_type in data.columns:
    earth = proportion[proportion["group"] == "earth"][cell_type]
    space = proportion[proportion["group"] == "space"][cell_type]
    ttest = ttest_ind(earth, space)
    print(f"{cell_type}: {ttest.pvalue.round(4)}")

<span style="color:blue; font-weight:bold; font-size:24px">Bonus Question. So, did space travel led to differential abundance of any of the immune cell types?</span>

# Appendix: Code for performing the pre-processing of the data

Students are not required to run this code - this was done before the workshop to prepare the data for the workshop and is provided here for reference.

```python
import scanpy as sc
# Read the unprocessed data
adata = sc.read_h5ad("data/mouse_spleen.h5ad")
# mitochondrial genes starts with "mt-" for mouse
adata.var["mt"] = adata.var_names.str.startswith("mt-")
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], inplace=True, log1p=True)
# filter cells if they do not express at least 200 genes
sc.pp.filter_cells(adata, min_genes=200)
# filter genes if they are expressed in at least 3 cells
sc.pp.filter_genes(adata, min_cells=3)
# Normalise (library-size correct) the data matrix 𝐗 to 10,000 counts per cell, so that information become comparable between cells.
sc.pp.normalize_total(adata, target_sum=1e4)
# Logarithmise the data:
sc.pp.log1p(adata)
# Identify highly variable genes
sc.pp.highly_variable_genes(adata)
# stash the normalised counts in .raw
adata.raw = adata
# Regress out the effects of total counts per cell and the percentage of mitochondrial genes expressed
sc.pp.regress_out(adata, ["total_counts", "pct_counts_mt"])
# Scale the data
sc.pp.scale(adata, max_value=10)
# Perform PCA
sc.tl.pca(adata, svd_solver="arpack", n_comps=80)
# Compute the neighborhood graph of cells
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=80)
# Compute the UMAP
sc.tl.umap(adata, min_dist=0.3)
# Perform clustering
sc.tl.leiden(adata, resolution=1)
# Marker gene test
sc.tl.rank_genes_groups(adata, groupby="leiden")
# Make the data smaller before saving
adata = adata.raw.to_adata()
# Save the processed data
adata.write("data/mouse_spleen_processed.h5ad", compression="gzip")
```

# References

https://scanpy.readthedocs.io/en/stable/tutorials/index.html

https://www.sc-best-practices.org/preamble.html
